In [13]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import datetime

In [2]:
def trend_break(df, stop_date):
    counties = df['Unnamed: 3']
    date_lst = [i for i in df.iloc[:,5:] if 'Unnamed' not in i]
    tstpos_lst = [i for i in df.iloc[:,5:] if df[i][0][0:6] == 'tstpos']
    mort_lst = [i for i in df.iloc[:,5:] if df[i][0][0:4] == 'mort']
    dic = {}
    for i in range(len(tstpos_lst)):
        if i == 0:
            continue
        date = date_lst[i]  

        if date == stop_date:
            break
        curr_c = df[tstpos_lst[i]]
        prev_c = df[tstpos_lst[i-1]]
        curr_c = curr_c[1:].fillna(0).replace(',','', regex=True).apply(int)
        prev_c = prev_c[1:].fillna(0).replace(',','', regex=True).apply(int)
        diff_c = curr_c - prev_c
        neg_idx_c = np.where(diff_c < 0)[0]
        # neg_idx_c = diff_c.index[diff_c < 0]
        tb_counties_c = counties[neg_idx_c]
        
        
        curr_d = df[mort_lst[i]]
        prev_d = df[mort_lst[i-1]]
        curr_d = curr_d[1:].fillna(0).replace(',','', regex=True).apply(int)
        prev_d = prev_d[1:].fillna(0).replace(',','', regex=True).apply(int)
        diff_d = curr_d - prev_d
        neg_idx_d = np.where(diff_d < 0)[0]
        # neg_idx_d = diff_d.index[diff_d < 0]
        tb_counties_d = counties[neg_idx_d]

        state_cases = tb_counties_c[tb_counties_c.str.contains(',')].str.split(',').apply(lambda x: x[-1])
        state_deaths = tb_counties_d[tb_counties_d.str.contains(',')].str.split(',').apply(lambda x: x[-1])

        # replace with tb_counties_c, tb_counties_d to get the specific counties
        exceptions = ['Georgia','Indiana','Michigan','Connecticut','North Dakota','Virginia','Massachusetts','Maine','Texas','Vermont',
            'Tennessee','Pennsylvania','California','Puerto Rico','Minnesota','Washington','Wisconsin','Illinois','Ohio']
        out_pre = pd.concat([state_cases, state_deaths]) 
        out = []
        for i in out_pre:
            if i.strip() not in exceptions:
                out.append(i.strip())
        if len(out) > 0:
            dic[date] = set(out)
     
    return dic

In [3]:
# insert all sheet urls here
url_lst = ['https://docs.google.com/spreadsheets/d/1e_uZrmf7x5fRKpOAjho_jR8GyS1uYXOhLvtM-2iqKPE/edit#gid=1642263273',
'https://docs.google.com/spreadsheets/d/1dZHw73L7zHKurw-qe6WTwNUWo1sRtQTYSRgWa86vq48/edit#gid=1642263273',
'https://docs.google.com/spreadsheets/d/1z1-WK4PC7BXSIkAgbvNXtyXj8DZ2DmrC0-lVYuDb5H8/edit#gid=1642263273',
'https://docs.google.com/spreadsheets/d/1f1zrKYHHD_eGMMEbj_cqho9lxI6mTLcFAyTlLovCup0/edit#gid=1642263273',
'https://docs.google.com/spreadsheets/d/1-ldl2hBdKog95GqqrxTcNdmHtimAFg1LrNPVZKTyUUg/edit#gid=311031188',
'https://docs.google.com/spreadsheets/d/1vJG2UopyZIfDzLAPQ_jrb_WQ_I0lI3ACW0WbQ9HK-os/edit#gid=311031188'
]

regions = {'midwest': ['Nebraska', 'Iowa', 'Kentucky', 'Kansas', 'South Dakota'],
                    'mountains': ['Arizona', 'Idaho', 'Montana', 'New Mexico', 'Wyoming'],
                    'prairie_mrv': ['Oklahoma', 'Louisiana', 'Arkansas', 'Mississippi', 'Missouri'],
                    'northeast': ['Delaware', 'Maryland', 'New Hampshire', 'New York', 'Rhode Island', 'Washington', 'New Jersey'],
                    'southeast': ['Alabama', 'Florida', 'North Carolina', 'South Carolina', 'West Virginia'],
                    'w_pacific' : ['Nevada', 'Utah', 'Oregon', 'Alaska', 'Hawaii']
                    }

all_regions = []
for i in url_lst:
    sheet_url = i
    url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
    df = pd.read_csv(url_1)

    # insert date after the day you want to stop checking for trend breaks
    all_regions.append(trend_break(df, '10/16/2022'))

In [4]:
midwest = {}
mountains = {}
prairie_mrv = {}
northeast = {}
southeast = {}
w_pacific = {}

for sheet in all_regions:
    for date in sheet:
        for state in sheet[date]:
            for region in regions:
                if state in regions[region]:
                    curr = eval(region)
                    if date not in curr.keys():
                        curr[date] = [state]
                    else:
                        curr[date] += [state]

In [21]:
midwest = dict(OrderedDict(sorted(midwest.items(), key=lambda t: datetime.datetime.strptime(t[0], '%m/%d/%Y').date())))
midwest

{'9/13/2022': ['Nebraska', 'South Dakota'],
 '9/14/2022': ['Iowa'],
 '9/15/2022': ['Kentucky'],
 '9/19/2022': ['Kentucky'],
 '9/21/2022': ['Kansas'],
 '9/26/2022': ['South Dakota', 'Kentucky'],
 '9/27/2022': ['South Dakota'],
 '9/28/2022': ['Iowa'],
 '9/29/2022': ['South Dakota', 'Iowa', 'Kansas'],
 '10/5/2022': ['Iowa', 'Kansas'],
 '10/12/2022': ['Iowa'],
 '10/14/2022': ['South Dakota']}

In [22]:
mountains = dict(OrderedDict(sorted(mountains.items(), key=lambda t: datetime.datetime.strptime(t[0], '%m/%d/%Y').date())))
mountains

{'9/13/2022': ['New Mexico'],
 '9/20/2022': ['New Mexico'],
 '9/21/2022': ['Wyoming', 'New Mexico'],
 '9/22/2022': ['Idaho'],
 '9/23/2022': ['Montana', 'New Mexico'],
 '9/26/2022': ['New Mexico'],
 '9/28/2022': ['New Mexico'],
 '9/30/2022': ['Montana', 'New Mexico'],
 '10/3/2022': ['New Mexico'],
 '10/4/2022': ['Wyoming', 'New Mexico'],
 '10/7/2022': ['New Mexico'],
 '10/11/2022': ['New Mexico'],
 '10/12/2022': ['New Mexico', 'Arizona'],
 '10/14/2022': ['New Mexico']}

In [24]:
prairie_mrv = dict(OrderedDict(sorted(prairie_mrv.items(), key=lambda t: datetime.datetime.strptime(t[0], '%m/%d/%Y').date())))
prairie_mrv

{'9/15/2022': ['Louisiana'],
 '9/18/2022': ['Arkansas'],
 '9/20/2022': ['Arkansas'],
 '9/26/2022': ['Arkansas', 'Louisiana'],
 '9/28/2022': ['Arkansas', 'Louisiana'],
 '9/30/2022': ['Missouri'],
 '10/4/2022': ['Arkansas'],
 '10/10/2022': ['Louisiana'],
 '10/12/2022': ['Arkansas', 'Louisiana'],
 '10/13/2022': ['Louisiana', 'Oklahoma'],
 '10/14/2022': ['Arkansas']}

In [20]:
northeast = dict(OrderedDict(sorted(northeast.items(), key=lambda t: datetime.datetime.strptime(t[0], '%m/%d/%Y').date())))
northeast

{'9/13/2022': ['New Hampshire'],
 '9/14/2022': ['New Jersey'],
 '9/15/2022': ['New Jersey'],
 '9/16/2022': ['Delaware', 'New Jersey'],
 '9/17/2022': ['New Jersey'],
 '9/18/2022': ['New York', 'New Hampshire'],
 '9/19/2022': ['Rhode Island', 'New Jersey', 'Maryland'],
 '9/20/2022': ['Maryland', 'New Jersey'],
 '9/21/2022': ['New Jersey'],
 '9/22/2022': ['Maryland', 'New Jersey'],
 '9/23/2022': ['Maryland', 'New York', 'New Jersey', 'New Hampshire'],
 '9/26/2022': ['Delaware', 'New Jersey', 'Rhode Island'],
 '9/27/2022': ['New Jersey', 'New Hampshire'],
 '9/28/2022': ['New Hampshire'],
 '9/29/2022': ['Maryland', 'New York', 'New Jersey', 'New Hampshire'],
 '9/30/2022': ['Maryland', 'New Jersey', 'New Hampshire'],
 '10/2/2022': ['New York'],
 '10/3/2022': ['Maryland', 'New York', 'New Jersey'],
 '10/5/2022': ['Maryland', 'New Jersey'],
 '10/6/2022': ['Maryland', 'New Jersey'],
 '10/7/2022': ['New Hampshire'],
 '10/9/2022': ['New Jersey'],
 '10/10/2022': ['New York', 'New Jersey'],
 '10/11

In [19]:
southeast = dict(OrderedDict(sorted(southeast.items(), key=lambda t: datetime.datetime.strptime(t[0], '%m/%d/%Y').date())))
southeast

{'9/15/2022': ['West Virginia', 'North Carolina'],
 '9/28/2022': ['North Carolina'],
 '10/12/2022': ['West Virginia'],
 '10/13/2022': ['West Virginia'],
 '10/14/2022': ['West Virginia']}

In [18]:
w_pacific = dict(OrderedDict(sorted(w_pacific.items(), key=lambda t: datetime.datetime.strptime(t[0], '%m/%d/%Y').date())))
w_pacific

{'9/14/2022': ['Alaska'],
 '9/15/2022': ['Alaska'],
 '9/17/2022': ['Utah'],
 '9/21/2022': ['Utah', 'Oregon', 'Nevada'],
 '9/22/2022': ['Utah'],
 '9/28/2022': ['Utah', 'Nevada', 'Oregon'],
 '10/5/2022': ['Utah'],
 '10/10/2022': ['Nevada'],
 '10/12/2022': ['Utah', 'Oregon'],
 '10/14/2022': ['Nevada']}